In [130]:
from keys import client_id, client_secret, user_name
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
import random
from functools import reduce

In [131]:
scope = 'user-library-read playlist-modify-public playlist-read-private user-read-recently-played app-remote-control user-top-read'

redirect_uri = 'https://developer.spotify.com/dashboard/applications/0743a195f7654b5ab95560a95e89316a'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(user_name, scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("NO TOKEN FOUND")


In [132]:
user_top_tracks = sp.current_user_top_tracks(limit=50, offset=0, time_range='long_term')
song_data = user_top_tracks["items"]

song_ids = []
song_names = []

for i in range(0, len(song_data)):
    if song_data[i]['id'] != None:
        song_ids.append(song_data[i]['id'])
        song_names.append(song_data[i]['name'])

song_features = []
for i in range(0, len(song_ids)):
    features = sp.audio_features(song_ids[i])
    for song in features:
        song_features.append(song)

top_tracks_df = pd.DataFrame(song_features, index=song_names)
top_tracks_df = top_tracks_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]
top_tracks_df.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
HOLIDAY,6zFMeegAMYQo0mt8rXtrli,0.12000,0.810,154998,0.511,0.000000,5,0.0832,-6.924,0,0.1640,151.947,0.837
Death of a Bachelor,1BECwm5qkaBwlbfo4kpYx8,0.01370,0.462,203507,0.538,0.000000,0,0.4290,-5.527,1,0.0590,139.256,0.405
Daylight,6Ed1q0X8oSKSm4IIhiQbYg,0.00488,0.528,163906,0.749,0.000096,7,0.0949,-7.571,1,0.0479,163.944,0.729
Knock Knock,3uYm4MtU6jUQft2DtGqEoZ,0.03530,0.667,202807,0.922,0.000000,4,0.3640,-4.857,0,0.1280,108.952,0.509
Don't Matter,7I6DceMT3utDOHjcYCbrr4,0.24400,0.798,293053,0.443,0.000000,2,0.3260,-6.062,1,0.0403,125.232,0.354


In [133]:
top_tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, HOLIDAY to Team
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                50 non-null     object 
 1   acousticness      50 non-null     float64
 2   danceability      50 non-null     float64
 3   duration_ms       50 non-null     int64  
 4   energy            50 non-null     float64
 5   instrumentalness  50 non-null     float64
 6   key               50 non-null     int64  
 7   liveness          50 non-null     float64
 8   loudness          50 non-null     float64
 9   mode              50 non-null     int64  
 10  speechiness       50 non-null     float64
 11  tempo             50 non-null     float64
 12  valence           50 non-null     float64
dtypes: float64(9), int64(3), object(1)
memory usage: 5.5+ KB


In [134]:
featured_playlists = sp.featured_playlists(limit=50)
# Creating data frame for playlists
id = []
name = []
num_tracks = []
items = featured_playlists['playlists']['items']
for item in items:
    id.append(item["id"])
    name.append(item["name"])
    num_tracks.append(item["tracks"]["total"])

df_pl_data = pd.DataFrame({"id":id, "name": name, "num_tracks": num_tracks})

In [135]:
df_pl_data

,id,name,num_tracks
0,37i9dQZF1DWU0ScTcjJBdj,Relax & Unwind,100
1,37i9dQZF1DWTwbZHrJRIgD,Happy Weekend,57
2,37i9dQZF1DX4pUKG1kS0Ac,Guilty Pleasures,151
3,37i9dQZF1DWYAcBZSAVhlf,Walking On Sunshine,80
4,37i9dQZF1DWWBHeXOYZf74,POLLEN,139
5,37i9dQZF1DX4o1oenSJRJd,All Out 00s,100
6,37i9dQZF1DX4bSrsRWE9cd,Bliss,75
7,37i9dQZF1DWZKuerrwoAGz,Happy Favorites,100
8,37i9dQZF1DX4VvfRBFClxm,Acoustic Hits,100
9,37i9dQZF1DX4MTfCb9IRyO,Summertime Blues,50


In [136]:
# Returns Individual Tracks In Playlists
def get_playlist_tracks(sp, playlist_id):
    data_track = sp.playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None)['items']
    track_id = []
    track_name = []
    
    for data in data_track:
        track_id.append(data['track']['id'])
        track_name.append(data['track']['name'])

    return pd.DataFrame({"track_id":track_id, "track_name": track_name})

In [137]:
get_playlist_tracks(sp, '37i9dQZF1DWZKuerrwoAGz').head()

,track_id,track_name
0,6FE2iI43OZnszFLuLtvvmg,Classic
1,3DmW6y7wTEYHJZlLo1r6XJ,Shower
2,4E5P1XyAFtrjpiIxkydly4,Replay
3,3E7dfMvvCLUddWissuqMwr,Party In The U.S.A.
4,1CQ2cMfrmFM1YdfmjENKVE,She Looks So Perfect


In [138]:
# Function for getting audio features of individual tracks
def get_audio_features(sp, playlist_id):
    playlist = get_playlist_tracks(sp, playlist_id)
    audio_features = []
    for i in range(len(playlist)):
        song_data_id = playlist['track_id'][i]
        audio_features.append(sp.audio_features(song_data_id))
    features_array = []
    for features in audio_features:
        features = features[0]
        features_array.append([features['danceability'],
                              features['acousticness'],
                              features['energy'], 
                              features['tempo'],
                              features['instrumentalness'], 
                              features['loudness'],
                              features['liveness'],
                              features['duration_ms'],
                              features['key'],
                              features['valence'],
                              features['speechiness']
                             ])
    df_audio_features = pd.DataFrame(features_array, columns=['danceability', 'acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 'liveness', 'duration_ms', 'key', 'valence', 'speechiness'])
    # DataFrame with audio features and playlist id/name
    df_playlist_features = pd.concat([playlist, df_audio_features], axis=1)
    df_playlist_features.set_index('track_name', inplace=True, drop=True)
    return df_playlist_features

In [139]:
get_audio_features(sp, '37i9dQZF1DXcRXFNfZr7Tp').head()

,track_id,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness
track_name,,,,,,,,,,,,
Beautiful Mistakes (feat. Megan Thee Stallion),6fRxMU4LWwyaSSowV441IU,0.713,0.0377,0.676,99.048,0.000000,-5.483,0.154,227395,10,0.721,0.0270
Peaches (feat. Daniel Caesar & Giveon),4iJyoBOLtHqaGxP12qzhQI,0.677,0.3210,0.696,90.030,0.000000,-6.181,0.420,198082,0,0.464,0.1190
deja vu,61KpQadow081I2AsbeLcsb,0.439,0.5930,0.610,181.088,0.000011,-7.236,0.341,215508,9,0.172,0.1160
BED,7jXQUrVhEpXdymfFWNDnQW,0.663,0.0134,0.783,123.986,0.001790,-4.585,0.325,178088,6,0.622,0.0393
Met Him Last Night (feat. Ariana Grande),0BI0hfbmqybnd3TezrDME3,0.538,0.2200,0.512,144.978,0.000000,-4.548,0.101,204632,4,0.120,0.0262


In [140]:
# # Turning playlist data into DataFrames of audio features for each track using previous method

# for i in range(0, len(df_pl_data['id'])):
#     string_command = "df_{} = get_audio_features(sp, df_pl_data['id'][i])".format(df_pl_data['id'][i])
#     print("Created: df_{}".format(df_pl_data['id'][i]))
#     exec(string_command)
# df_37i9dQZF1DWU0ScTcjJBdj

In [141]:
# Finding mean of audio features for each playlist
def audio_features_mean(sp, playlist_id):
    Playlist = get_audio_features(sp, playlist_id)
    return pd.DataFrame(Playlist.mean(), columns= [playlist_id])

In [142]:
audio_features_mean(sp, '37i9dQZF1DXbYM3nMM0oPk')

,37i9dQZF1DXbYM3nMM0oPk
danceability,0.709173
acousticness,0.181992
energy,0.645507
tempo,121.424267
instrumentalness,0.008807
loudness,-5.947773
liveness,0.162463
duration_ms,188870.400000
key,5.253333
valence,0.545632


In [143]:
# Merges average of audio features of each playlist into a single DataFrame
primary_df = []
for i in range(len(df_pl_data)):
    primary_df.append(audio_features_mean(sp, df_pl_data['id'][i]))

TypeError: 'NoneType' object is not subscriptable

In [45]:
primary_df

[                  37i9dQZF1DWU0ScTcjJBdj
 danceability                    0.521320
 acousticness                    0.695853
 energy                          0.363995
 tempo                         117.174500
 instrumentalness                0.074661
 loudness                      -11.082280
 liveness                        0.155497
 duration_ms                241161.370000
 key                             5.810000
 valence                         0.297808
 speechiness                     0.038116,
                   37i9dQZF1DWTwbZHrJRIgD
 danceability                    0.682509
 acousticness                    0.281712
 energy                          0.677877
 tempo                         119.546456
 instrumentalness                0.027628
 loudness                       -7.268737
 liveness                        0.158749
 duration_ms                241177.192982
 key                             5.105263
 valence                         0.577068
 speechiness                     

In [49]:
# Turns playlist feature mean data into correctly formatted DataFrame
X_data = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), primary_df)
X_data

,37i9dQZF1DWU0ScTcjJBdj,37i9dQZF1DWTwbZHrJRIgD,37i9dQZF1DX4pUKG1kS0Ac,37i9dQZF1DWYAcBZSAVhlf,37i9dQZF1DWWBHeXOYZf74,37i9dQZF1DX4o1oenSJRJd,37i9dQZF1DX4bSrsRWE9cd,37i9dQZF1DWZKuerrwoAGz,37i9dQZF1DX4VvfRBFClxm,37i9dQZF1DX4MTfCb9IRyO
danceability,0.521320,0.682509,0.670200,0.709537,0.658740,0.698180,0.493147,0.698110,0.577720,0.593440
acousticness,0.695853,0.281712,0.098756,0.171681,0.271123,0.096046,0.816849,0.090534,0.656251,0.506314
energy,0.363995,0.677877,0.781580,0.783838,0.595230,0.753570,0.235191,0.783260,0.369526,0.494940
tempo,117.174500,119.546456,122.607870,124.145737,116.348440,121.188730,118.281827,118.099400,117.764690,114.266500
instrumentalness,0.074661,0.027628,0.010865,0.025119,0.099894,0.018002,0.162121,0.004267,0.001867,0.086893
loudness,-11.082280,-7.268737,-5.103840,-8.675512,-8.161440,-5.191770,-14.750040,-4.660960,-8.558720,-10.999140
liveness,0.155497,0.158749,0.176189,0.194721,0.185388,0.177294,0.132576,0.180894,0.139820,0.189042
duration_ms,241161.370000,241177.192982,219849.220000,243473.900000,208252.740000,230006.420000,231474.000000,214034.980000,211450.860000,219416.400000
key,5.810000,5.105263,5.280000,4.875000,5.570000,5.390000,3.813333,5.210000,5.310000,5.400000
valence,0.297808,0.577068,0.689600,0.818263,0.508524,0.638300,0.249524,0.660010,0.416764,0.741960


In [21]:
# Turns my favorite track data (Y_data) into a DataFrame with audio features averaged
# maybe change mean to median
Y_data = pd.DataFrame(top_tracks_df.median(), columns=['top_tracks']).drop('mode')
Y_data

,top_tracks
acousticness,0.03825
danceability,0.65550
duration_ms,204280.00000
energy,0.70150
instrumentalness,0.00000
key,7.00000
liveness,0.12750
loudness,-5.16550
speechiness,0.06010
tempo,120.64150


In [128]:
# Model for predicting playlist of "best-fit" using feature_importances_
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=44, max_depth=5, max_features=8, oob_score=True)
model.fit(X_data, Y_data['top_tracks'])
pl_rank = model.feature_importances_
pl_importances = pd.DataFrame(pl_rank, index = X_data.columns, columns=['importance']).sort_values('importance',                                        ascending=False)

pl_importances

,importance
37i9dQZF1DWZKuerrwoAGz,0.171458
37i9dQZF1DX4pUKG1kS0Ac,0.146059
37i9dQZF1DX4o1oenSJRJd,0.126806
37i9dQZF1DX4bSrsRWE9cd,0.119544
37i9dQZF1DX4MTfCb9IRyO,0.089318
37i9dQZF1DX4VvfRBFClxm,0.084949
37i9dQZF1DWYAcBZSAVhlf,0.078111
37i9dQZF1DWWBHeXOYZf74,0.075053
37i9dQZF1DWU0ScTcjJBdj,0.067547
37i9dQZF1DWTwbZHrJRIgD,0.041154


In [129]:
pl_t3 = pl_importances.index[0:3]
frames = []
for i in range(len(pl_t3)):
    frame = get_audio_features(sp, pl_t3[i])
    print(len(frame))
    frames.append(frame)
recommended_pl = pd.concat(frames)
recommended_pl

HTTP Error for GET to https://api.spotify.com/v1/playlists/37i9dQZF1DWZKuerrwoAGz/tracks returned 401 due to The access token expired


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/playlists/37i9dQZF1DWZKuerrwoAGz/tracks?limit=100&offset=0&additional_types=track:
 The access token expired, reason: None